<a href="https://colab.research.google.com/github/gothammered/2022-2-Python/blob/main/HW3_SubwayNavigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
"""
Using xlrd is a good way to read excel file.
However, because not only the package is no longer maintained but also can only read xls file which is also deprecated, this code used pandas instead.
(Please refer to https://github.com/python-excel/xlrd for further information regarding xlrd package)
"""

# Instead of
# import xlrd
# this code will
import pandas as pd
# because of the reasons mentioned above.

# in order to break code,
import sys

# to hash password input,
import hashlib

# In reality, it is NOT safe to store salt info inside a same place
# However, for easier implementation, salt will be defined here
salt = 'HelloWorld'
# Of course, in reality, salt string should NOT be this obvious :)

"""
Back-end(?) Part
"""
# 0. Data load and preprocessing
# 1) read excel file
print('Loading...\n')
df_en = pd.read_excel('./simplified_subway_info_english.xlsx')
df_ko = pd.read_excel('./simplified_subway_info_korean.xlsx')

# 2) Convert each columns to list, as shown in the provided sample codes.
#    For loop is required to do this because the pandas package does not provide such exact function.

# 2-1) Retrieve a list of column names for each DataFrame
df_en_columnList = df_en.columns.tolist()
df_ko_columnList = df_ko.columns.tolist()

# 2-2) Retrieve a list of station names for each columns from respective DataFrame
#      (Optional) We can use tqdm function from tqdm package to see how much iterations have passed in for loop
from tqdm import tqdm

"""
DEPRECATED (HW1)
# var list should be declared global
data_en = []
data_ko = []

# NAs must be dropped before converting to list
for line in tqdm(df_en_columnList, desc='Loading information(EN)...'):
    data_en.append(df_en[line].dropna().tolist())

for line in tqdm(df_ko_columnList, desc='Loading information(KO)...'):
    data_ko.append(df_ko[line].dropna().tolist())

# change station names in data_en to uppercase and save it to data_en_query
data_en_query = []
for stations in data_en:
    data_en_query.append([x.upper() for x in stations])
# data_ko_query is the same as data_ko, but create one for better readability
data_ko_query = data_ko     # deepcopy should be used later if different items will be saved in the two lists
"""

# Using dictionary instead of list is required in HW2
# var dictionary should be declared global
data_en = {}
data_ko = {}

# NAs must be dropped before converting to dictionary
for line in tqdm(df_en_columnList, desc='Loading information(EN)...'):
    data_en[line] = tuple(df_en[line].dropna().values)    # {'1호선': ('Soyosan', 'Dongducheon', ...), '2호선': (...)}

for line in tqdm(df_ko_columnList, desc='Loading information(KO)...'):
    data_ko[line] = tuple(df_ko[line].dropna().values)    # {'1호선': ('소요산', '동두천', ...), '2호선': (...)}

# change station names in data_en to uppercase and save it to data_en_query
data_en_query = {}
for line in df_en_columnList:
    data_en_query[line] = tuple([x.upper() for x in list(data_en[line])])
# data_ko_query is the same as data_ko, but create one for better readability
data_ko_query = data_ko     # deepcopy should be used later if different items will be saved in the two lists

print(data_en_query)
print(data_ko_query)

# create a dictionary containing transfer station information
data_transfer_en = {}
data_transfer_ko = {}

for lineNum_O in tqdm(range(1, 5), desc='Loading transfer information(EN)...'):    # {'1-2': ['City hall', 'Sindorim'], '1-3': ['...', ], ...}
    for lineNum_D in range(1, 5):
        transfer_list = []
        for station in data_en['{0}호선'.format(lineNum_O)]:
            if lineNum_O == lineNum_D:
                pass
            else:
                if station in data_en['{0}호선'.format(lineNum_D)]:
                    transfer_list.append(station)
                
                data_transfer_en['{0}-{1}'.format(lineNum_O, lineNum_D)] = transfer_list


for lineNum_O in tqdm(range(1, 5), desc='Loading transfer information(KO)...'):    # {'1-2': ['시청', '신도림'], '1-3': ['...', ], ...}
    for lineNum_D in range(1, 5):
        transfer_list = []
        for station in data_ko['{0}호선'.format(lineNum_O)]:
            if lineNum_O == lineNum_D:
                pass
            else:
                if station in data_ko['{0}호선'.format(lineNum_D)]:
                    transfer_list.append(station)
                
                data_transfer_ko['{0}-{1}'.format(lineNum_O, lineNum_D)] = transfer_list

# Define user class
class User:
    def __init__(self):
        self.username = 'username'
        self.password = 'password'
        self.recentRequest_O = list()
        self.recentRequest_D = list()

    def appendRecord(self, station_O, station_D):
        self.recentRequest_O = [station_O] + self.recentRequest_O
        self.recentRequest_D = [station_D] + self.recentRequest_D

# User info will be saved in sqlite db
from sqlite3 import connect

# Load user info DataFrame
try:
    userInfoDF = pd.read_excel('./userInfo.xlsx', dtype={'recentRequest_O': 'object', 'recentRequest_D': 'object'})
    userInfoDF = userInfoDF.rename(columns={'Unnamed: 0': 'username'})
    userInfoDF = userInfoDF.set_index('username', drop=True)

except:
    print('userInfo not found... creating empty DataFrame')
    userInfoDF = pd.DataFrame()

if ('recentRequest_O' in userInfoDF.columns.tolist()) and ('recentRequest_D' in userInfoDF.columns.tolist()):
    # string to list
    for col in ['recentRequest_O', 'recentRequest_D']:
        userInfoDF[col] = userInfoDF[col].str.replace('[', '')
        userInfoDF[col] = userInfoDF[col].str.replace(']', '')
        userInfoDF[col] = userInfoDF[col].str.replace("'", "")
        userInfoDF[col] = userInfoDF[col].str.replace('"', '')
        userInfoDF[col] = userInfoDF[col].apply(lambda x: x.split(', '))


# print blank row for better readability
print('\n')
print('Program loading successful!')
print('\n')


"""
Front-end(?) part
"""
# login function
def login():
    print('*********' * 10)
    print('Please select an option...')
    print('1. Log in (or Sign Up)')
    print('2. Exit')
    print('*********' * 10)

    # Get user input(choose menu)
    opt = input('Please choose one of the options (1 - 2): ')

    # if blank, run the query again
    if len(opt) == 0:
        print('Nothing was entered... Please try again...')
        login()
        return
    
    # if entered 1, run loginQuery()
    elif opt == '1':
        print('\nLoading login interface...')
        usernameQuery()
        return
    
    # if entered 2, run askSure()
    elif opt == 2:
        askSure(login)
        return

    # if no such option exist, ask again
    else:
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
        login()
        return

# loginQuery function
def usernameQuery(fromBeginning=True, username=None):
    # if fromBeginning, start from beginning
    if fromBeginning:
        print('*********' * 10)
        print('Log in (or Sign Up)')
        print('*********' * 10)
        username = input('Please enter username: ')
    
    # if blank, run the query again
    if len(username) == 0:
        print('Noting was entered... Please try again...')
        usernameQuery()
        return
    
    # if the username is not in username list, ask if the user wants to create one or not
    elif username not in userInfoDF.index.tolist():
        print('*********' * 10)
        print('Username "{0}" not found... Do you want to sign up?'.format(username))
        print('*********' * 10)
        respond = input('(Y / N): ')

        # if blank, run the query again
        if len(respond) == 0:
            print('Nothing was entered... Please try again...')
            usernameQuery(False, username)
            return

        # if Y or y, run createUser()
        elif respond == 'Y' or respond == 'y':
            createUser()
            return

        # if N or n, return to loginQuery()
        elif respond == 'N' or respond == 'n':
            usernameQuery()
            return

        # else, run the query again
        else:
            print('\nThe option "{0}" does not exist... Please try again...'.format(respond))
            usernameQuery(False, username)
            return

    # if the username exist, ask password
    else:
        passwordQuery(username)
        

def passwordQuery(username):
    sha = hashlib.new('sha256')
    password_input = input('Please enter password: ')    
    
    # if blank, run query again from usernameQuery
    if len(password_input) == 0:
        print('Nothing was entered... Please try again...')
        usernameQuery()
        return

    else:
        sha.update((password_input + salt).encode())
        password_input = '' # Clear user password input from memory to prevent any security accidents
        password_actual = userInfoDF.at[username, 'password']
        
        if sha.hexdigest() == password_actual:
            user = User()
            user.username = username
            # recentRequest might not exist
            try:
                user.recentRequest_O = userInfoDF.at[username, 'recentRequest_O']
                user.recentRequest_D = userInfoDF.at[username, 'recentRequest_D']
            except:
                user.recentRequest_O = list()            
                user.recentRequest_D = list()

            main(user)
            return

        else:
            print('Password is incorrect... Please try again')
            passwordQuery(username)
            return


# create userfunction
def createUser():
    print('*********' * 10)
    print('Sign up')
    print('*********' * 10)
    username = input('Please enter username: ')

    # if blank, run the query again
    if len(username) == 0:
        print('Nothing was entered... Please try again...')
        createUser()
        return

    # else, ask password
    else:
        createPassword(username)
        return

def createPassword(username, salt=salt):
    # It is SUICIDAL to save password in plain text, so use SHA-256 and salt to hash
    sha = hashlib.new('sha256')
    password = input('Please enter password: ')
    
    # if blank, ask again
    if len(password) == 0:
        print('Nothing was entered... Please try again...')
        createPassword(username)
        return

    else:
        sha.update((password + salt).encode())
        userInfoDF.at[username, 'password'] = sha.hexdigest()
        userInfoDF.to_excel('./userInfo.xlsx')  # In reality, using a proper database is always better than using excel
        print('User created successfully! Please log in')
        login()


# main function
def main(user):
    print('*********' * 10)
    print('Welcome, {0}'.format(user.username))
    print('Please select an option...')
    print('1. Display subway line information (Line 1 - 4)')
    print('2. Display subway station information')
    print('3. Find a path between two subway stations')
    print('4. View my recent request')
    print('5. Exit')
    print('*********' * 10)

    # Get user input(choose menu)
    opt = input('Please choose one of the options (1 - 4): ')
    
    # print blank row for better readability
    print('\n')    

    # Error exception
    # The best is using try except, but I deliberately avoided using it
    # 49, 50, 51 are ASCII Decimal codes for '1', '2', '3', respectively
    # get only the first letter from string to avoid error from ord()
    # if blank, run the query again
    if len(opt) == 0:
        print('Nothing was entered... Please try again...')
        main(user)
        return
    
    # if entered 1, run subwayLineQuery()
    elif (len(opt)==1 and ord(opt[0])==49):
        print('\nRunning option 1...')
        subwayLineQuery(user)
        return

    # if entered 2, run stationNameQuery()
    elif (len(opt)==1 and ord(opt[0])==50):
        print('\nRunning option 2...')
        stationNameQuery(user)
        return

    # if entered 3, run pathQuery()
    elif (len(opt)==1 and ord(opt[0])==51):
        print('\n')
        pathQuery(user, showEntry=True)
        return

    # if entered 4, run recentRequestQuery()
    elif (len(opt)==1 and ord(opt[0])==52):
        print('\n')
        recentRequestQuery(user)
        return

    # if entered 5, run askSure()
    elif (len(opt)==1 and ord(opt[0])==53):
        print('\n')
        askSure(main, arg=user)
    
    # if no such option exist, ask again   
    else:
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
        print(ord(opt[0]))
        main()
        return


# When selected option 1 (Line information)
def subwayLineQuery(user):
    print('*********' * 10)
    print('Subway line information service')
    print('*********' * 10)
    lineNum = input('Please enter a subway line number (1 - 4), or enter 0 to return: ')

    # print blank row for better readability
    print('\n')

    # Error exception
    # The best is using try except, but I deliberately avoided using it as the same reason as above
    # if blank, run the query again 
    if len(lineNum) == 0:
        print('Nothing was entered... Please try again...')
        subwayLineQuery(user)
        return
    
    # if entered 0, return to main()
    if (len(lineNum)==1 and ord(lineNum[0]) == 48):
        main(user)
        return

    # if entered 1 through 4, show the line information through displayLineInfo()
    elif (len(lineNum)==1 and ord(lineNum[0]) in [49, 50, 51, 52]):
        displayLineInfo(user, int(lineNum))
        return

    # if no such option exist, ask again
    else:
        print('\nThe line "{0}" does not exist... Please try again...'.format(lineNum))
        subwayLineQuery(user)
        return

# When selected option 2 (Station information)
def stationNameQuery(user):
    print('*********' * 10)
    print('Subway station information service')
    print('*********' * 10)
    stationName = input('Please enter a subway station name, or enter 0 to return: ')

    # print blank row for better readability
    print('\n')

    # Error exception
    # The best is using try except, but I deliberately avoided using it as the same reason as above
    # if blank, run the query again
    if len(stationName) == 0:
        print('Nothing was entered... Please try again...')
        stationNameQuery(user)
        return

    # if entered 0, return to main()
    elif (len(stationName)==1 and ord(stationName[0]) == 48):
        main(user)
        return

    # else, toss the stationName to displayStationInfo() and find it there
    else:
        displayStationInfo(user, stationName)
        return

# When selected option 3 (Pathfind)
def pathQuery(user, station_O=None, showEntry=False):
    # Show entry only when came from main()
    if showEntry:
        print('*********' * 10)
        print('Subway navigation service')
        print('*********' * 10)

    # These two vars should be declared outside if-else
    station_O_tp = station_O
    line_O = []

    if station_O == None:
        station_O = input('Please enter origin subway station name, or enter 0 to return: ')
        # Error exception
        # The best is using try except, but I deliberately avoided using it as the same reason as above
        # if blank, run the query again
        if len(station_O) == 0:
            print('\nNothing was entered... Please try again...')
            pathQuery(user)
            return

        # if entered 0, return to main()
        elif (len(station_O)==1 and ord(station_O[0]) == 48):
            main(user)
            return

        # else, toss the stationName to displayStationInfo(isQuery=True) and investigate wheter the station exists or not
        else:
            existStation_O = displayStationInfo(user, station_O, isQuery=True)
            
            if not existStation_O:
                print('\nNo such station was found... Please try again...')
                pathQuery(user)
                return

            else:
                station_O_tp = station_O
                line_O = existStation_O

    else:
        line_O = displayStationInfo(user, station_O, isQuery=True)


    station_D = input('Please enter destination subway station name, or enter 0 to change origin subway station: ')

    # Error exception
    # The best is using try except, but I deliberately avoided using it as the same reason as above
    # if blank, run the query again
    if len(station_D) == 0:
        print('\nNothing was entered... Please try again...')
        pathQuery(user, station_O=station_O_tp)    # In this case, we do not need to ask the origin station name again
        return

    # if entered 0, ask origin subway station name again
    elif (len(station_O)==1 and ord(station_O[0]) == 48):
        pathQuery(user)
        return

    # if station_O == station_D, ask again
    elif station_O == station_D:
        print('\nThe destination station is the same as the origin... Please try again...')
        pathQuery(user, station_O=station_O_tp)    # In this case, we do not need to ask the origin station name again
        return

    # else, toss the stationName to displayStationInfo(isQuery=True) and investigate wheter the station exists or not
    else:
        existStation_D = displayStationInfo(user, station_D, isQuery=True)
        
        if not existStation_D:
            print('\nNo such station was found... Please try again...')
            pathQuery(user, station_O=station_O_tp)    # In this case, we do not need to ask the origin station name again
            return

        else:
            # Add query record to userInfoDF
            user.appendRecord(station_O, station_D)
            userInfoDF.at[user.username, 'recentRequest_O'] = user.recentRequest_O
            userInfoDF.at[user.username, 'recentRequest_D'] = user.recentRequest_D
            userInfoDF.to_excel('./userInfo.xlsx')
            displayPath(user, station_O, line_O, station_D, existStation_D)
            return


# When selected option 4 (Show recent requests)
def recentRequestQuery(user):
    print('*********' * 10)
    print("{0}'s recent requests service".format(user.username))
    print('*********' * 10)
    
    # In some cases user.recentRequest might not exist
    try:
        recentRequestCount = len(user.recentRequest_O)
    except:
        recentRequestCount = 0

    # if there is any recent request
    if recentRequestCount != 0:
        print('Your most recent navigation request: ')
        i = 1
        # 
        for o, d in zip(user.recentRequest_O, user.recentRequest_D):
            print('{0}. From {1} to {2}'.format(i, o, d))
            i += 1
        print('{0}. Go back'.format(i))

        opt = input('Please choose one of the options(1 - 2): ')
        
        # if option is not number
        try:
            opt_int = int(opt)      
        except:
            print('Please input a vaild number...')
            recentRequestQuery(user)
            return

        # if such option exists
        if opt_int in range(1, i):
            station_O, station_D = user.recentRequest_O[opt_int-1], user.recentRequest_D[opt_int-1]
            displayPath(user, station_O, displayStationInfo(user, station_O, isQuery=True), station_D, displayStationInfo(user, station_D, isQuery=True))
            return

        # if selected return
        elif opt_int == i:
            main(user)
            return

        # if no such option exists
        else:
            print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
            recentRequestQuery(user)
            return

    # if there is no recent request
    else:
        print('There are no navigation request record...')
        print('Going back to main menu')
        main(user)
        return
        
    

# Now that we have station list, 
# 1. Display subway line information
def displayLineInfo(user, lineNum, OD=None):    # == dispSubwayLineInfo()
    # We could just use
    # print(data_en[0])
    # print(data_ko[0])
    # for simple implementation, but I used zip for better readability
    
    # if OD == None, show all the stations in the line
    if OD == None:
        for ko, en in zip(data_ko['{0}호선'.format(lineNum)], data_en['{0}호선'.format(lineNum)]):    # changed data format from list to dictionary
            print(ko, en, sep='\t')

        # print blank row for better readability
        print('\n')

        # When done printing, ask user whether to return to main or not
        askReturn(main, subwayLineQuery, arg=user)
        return

    # elif OD != None, show the list of stations between the OD
    else:
        # If input is 'SEOUL', find 'SEOUL STATION' instead, if '서울', find '서울역' instead
        if OD[0].upper() == 'SEOUL':
            OD[0] = 'SEOUL STATION'
        if OD[1].upper() == 'SEOUL':
            OD[1] = 'SEOUL STATION'
        if OD[0] == '서울':
            OD[0] = '서울역'
        if OD[1] == '서울':
            OD[1] = '서울역'

        # if english, find from english
        if OD[0].upper() != OD[0].lower():
            O_idx = data_en_query['{0}호선'.format(lineNum)].index(OD[0].upper())
        else:
            O_idx = data_ko_query['{0}호선'.format(lineNum)].index(OD[0])
        
        if OD[1].upper() != OD[1].lower():
            D_idx = data_en_query['{0}호선'.format(lineNum)].index(OD[1].upper())
        else:
            D_idx = data_ko_query['{0}호선'.format(lineNum)].index(OD[1])

        # if going forward, just return
        if O_idx < D_idx:
            return zip(data_ko['{0}호선'.format(lineNum)][O_idx:D_idx+1], data_en['{0}호선'.format(lineNum)][O_idx:D_idx+1])
        
        # if going backward, return reversed zip
        else:
            O_idx, D_idx = D_idx, O_idx
            return zip(list(data_ko['{0}호선'.format(lineNum)][O_idx:D_idx])[::-1], list(data_en['{0}호선'.format(lineNum)][O_idx:D_idx])[::-1])
        

# 2. Display station information
def displayStationInfo(user, stationName, isQuery=False):    # == dispSubwayStationInfo()
    # First, change all the lower cases from input to upper cases since Python distinguishes the two
    # (User might not always input exact station names)
    stationNameToFind = stationName.upper()

    # If input is 'SEOUL', find 'SEOUL STATION' instead, if '서울', find '서울역' instead
    if stationNameToFind == 'SEOUL':
        stationNameToFind = 'SEOUL STATION'
    elif stationNameToFind == '서울':
        stationNameToFind = '서울역'
    
    # Then, try find the stationName from query lists, save line numbers to lineList
    # lineList should be declared outside the loop
    lineList = []
    for line in range(1, 5):
        if stationNameToFind in data_ko_query['{0}호선'.format(line)]:
            lineList.append(line)
        if stationNameToFind in data_en_query['{0}호선'.format(line)]:
            lineList.append(line)

    # if isQuery, return whether the station exists or not
    if isQuery:
        if len(lineList) == 0:
            return False
        else:
            return lineList
    
    else:
        # if lineList is blank, print that no such station name was found
        if len(lineList) == 0:
            print('No such station name was found... Please try again')
            stationNameQuery()
            return

        # if lineList is not blank, print line numbers
        # if only one line was found, print one line(without 'and')
        elif len(lineList) == 1:
            print('{0} station is in Line {1}'.format(stationName, lineList[0]))

        # if multiple lines were found, print those lines with sep 'and'
        else:
            lines = ''
            for line in lineList:
                lines += '{0} and '.format(line)
            
            # delete the last ' and ' from lines
            lines = lines[:-5]
            print('{0} station is in Line {1}'.format(stationName, lines))

        # print blank row for better readability
        print('\n')
        
        # When done printing, ask user whether to return to main or not
        askReturn(main, stationNameQuery, arg=user)
        return


# 3. Path navigation
def displayPath(user, station_O, line_O, station_D, line_D):
    # First, check whether the two stations are in the same line
    isInSameLine = False
    commonLines = []
    for line in line_O:
        if line in line_D:
            isInSameLine = True
            commonLines.append(line)

    # If they are in the same line, simply show the list of stations between the two stations
    if isInSameLine:
        pathLen = 0
        commonLine = 0
        for line in commonLines:
            # get length of each possible route
            stations_ko, stations_en = zip(*displayLineInfo(user, line, [station_O, station_D]))
            newPathLen = len(stations_ko)
            
            # save the result of the first query as a reference
            if pathLen == 0:
                pathLen = newPathLen
                commonLine = line
            
            # If the new result is better, change the reference to the new one
            elif pathLen > newPathLen:
                pathLen = newPathLen
                commonLine = line

            # If the old result is better, do not change the reference
            else:
                pass

        # display the best route
        info = displayLineInfo(user, commonLine, [station_O, station_D])

        # print outline of the route
        print('\nPrinting route from ({0}) to ({1})'.format(station_O, station_D))
        print('No transfer is required\n')

        # print the line to ride
        print('\nTAKE LINE no. ({0}) at ({1}) Station\n'.format(commonLine, station_O))
        
        for ko, en in info:    # changed data format from list to dictionary
            print(ko, en, sep='\t')

        # print blank row for better readability
        print('\nARRIVING at ({0}) Station\n'.format(station_D))
             
        # When done printing, ask user whether to return to main or not
        askReturn(main, pathQuery, arg=user)
        return

    # Else, show the list of stations between station_O and transfer station, transfer station and station_D
    else:
        pathLen = 0
        pathLine_O = 0
        pathLine_D = 0
        transferSt = ''
        for line_o in line_O:
            for line_d in line_D:
                for transferStation in data_transfer_en['{0}-{1}'.format(line_o, line_d)]:
                    stations_ko_O, stations_en_O = zip(*displayLineInfo(user, line_o, [station_O, transferStation]))
                    stations_ko_D, stations_en_D = zip(*displayLineInfo(user, line_d, [transferStation, station_D]))
                    newPathLen = len(stations_ko_O) + len(stations_ko_D)
                    # save the result of the first query as a reference
                    if pathLen == 0:
                        pathLen = newPathLen
                        pathLine_O = line_o
                        pathLine_D = line_d
                        transferSt = transferStation
                    
                    # If the new result is better, change the reference to the new one
                    elif pathLen > newPathLen:
                        pathLen = newPathLen
                        pathLine_O = line_o
                        pathLine_D = line_d
                        transferSt = transferStation

                    # If the old result is better, do not change the reference
                    else:
                        pass

        # display the best route
        info_O = displayLineInfo(user, pathLine_O, [station_O, transferSt])
        info_D = displayLineInfo(user, pathLine_D, [transferSt, station_D])
        
        # print outline of the route
        print('\nPrinting route from ({0}) to ({1})'.format(station_O, station_D))
        print('Transfer is required at ({0}) Station'.format(transferSt))
        
        # print the line to ride
        print('\nTAKE LINE no. ({0}) at ({1}) Station\n'.format(pathLine_O, station_O))

        for ko, en in info_O:    # changed data format from list to dictionary
            print(ko, en, sep='\t')

        # print that transfer is needed)
        print('\nTRANSFER to LINE no. ({0}) at ({1}) Station\n'.format(pathLine_D, transferSt))

        for ko, en in info_D:    # changed data format from list to dictionary
            print(ko, en, sep='\t')

        # print blank row for better readability
        print('\nARRIVING at ({0}) Station\n'.format(station_D))

        # When done printing, ask user whether to return to main or not
        askReturn(main, pathQuery, arg=user)
        return



# Used when to ask return
# Should be used for functions with SINGLE argument... further implementation is required for multible arguments
def askReturn(fun_opt1, fun_opt2, arg=None, desc_ask='Return to main menu?', desc_opt1='Yes', desc_opt2='No'):
    # Ask whether the user wants to return or stay
    # Descriptions could be altered for further implementation
    print('*********' * 10)
    print(desc_ask)
    print('1. {0}'.format(desc_opt1))
    print('2. {0}'.format(desc_opt2))
    print('*********' * 10)
    opt = input('Please choose one of the options (1 - 2): ')
    
    # print blank row for better readability
    print('\n')
    
    # Error exception
    if (len(opt)==1 and ord(opt[0]) == 49):
        # A function can be called like this in python
        # THANK YOU PYTHON
        if arg == None:
            fun_opt1()
            return
        else:
            fun_opt1(arg)
            return
    
    elif (len(opt)==1 and ord(opt[0]) == 50):
        # THANK YOU PYTHON, AGAIN
        # If no argument is declared, run the latter function with no arguments
        if arg == None:
            fun_opt2()
            return
        # Else, put argument to re-play the latter function
        else:
            fun_opt2(arg)
            return

    else:
        # If no such option exists, ask the user again
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
        askReturn(fun_opt1, fun_opt2, arg, desc_ask, desc_opt1, desc_opt2)
        return

# Used when to ask sure to exit
def askSure(before, arg=None):
    print('*********' * 10)
    print('Do you really want to exit the program?')
    print('1. Yes')
    print('2. No')
    print('*********' * 10)
    opt_exit = input('Please choose one of the options (1 - 2): ')

    # print blank row for better readability
    print('\n')

    # Error exception
    if (len(opt_exit)==1 and ord(opt_exit[0]) == 49):
        print('\nExiting the program... Thank you for using!')
        sys.exit()

    # If the user input 2, return to former function
    elif (len(opt_exit)==1 and ord(opt_exit[0]) == 50):
        print('\nReturning...')
        if arg == None:
            before()
        else:
            before(arg)
        return

    else:
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt_exit))
        askSure(before, arg)
        return


# RUN!
login()

Loading...



Loading information(KO)...: 100%|██████████| 4/4 [00:00<00:00, 2045.50it/s]


{'1호선': ('SOYOSAN', 'DONGDUCHEON', 'BOSAN', 'DONGDUCHEON JUNGANG', 'JIHAENG', 'DEOKJEONG', 'DEOKGYE', 'YANGJU', 'NOGYANG', 'GANEUNG', 'UIJEONGBU', 'HOERYONG', 'MANGWOLSA', 'DOBONGSAN', 'DOBONG', 'BANGHAK', 'CHANG-DONG', 'NOKCHEON', 'WOLGYE', 'KWANGWOON UNIV.', 'SEOKGYE', 'SINIMUN', 'HANKUK UNIV. OF FOREIGN STUDIES', 'HOEGI', 'CHEONGNYANGNI', 'JEGI-DONG', 'SINSEOL-DONG', 'DONGMYO', 'DONGDAEMUN', 'JONGNO 5(O)-GA', 'JONGNO 3(SAM)-GA', 'JONGGAK', 'CITY HALL', 'SEOUL STATION', 'NAMYEONG', 'YONGSAN', 'NORYANGJIN', 'DAEBANG', 'SINGIL', 'YEONGDEUNGPO', 'SINDORIM', 'GURO', 'GUIL', 'GAEBONG', 'ORYU-DONG', 'ONSU', 'YEOKGOK', 'SOSA', 'BUCHEON', 'JUNG-DONG', 'SONGNAE', 'BUGAE', 'BUPYEONG', 'BAEGUN', 'DONGAM', 'GANSEOK', 'JUAN', 'DOHWA', 'JEMULPO', 'DOWON', 'DONGINCHEON', 'INCHEON'), '2호선': ('SINDORIM', 'DAERIM', 'GURO DIGITAL COMPLEX', 'SINDAEBANG', 'SILLIM', 'BONGCHEON', 'SEOUL NAT`L UNIV.', 'NAKSEONGDAE', 'SADANG', 'BANGBAE', 'SEOCHO', 'SEOUL NAT`L UNIV. OF EDUCATION', 'GANGNAM', 'YEOKSAM', 'SEOL

Loading transfer information(KO)...: 100%|██████████| 4/4 [00:00<00:00, 2051.76it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:145: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.




Program loading successful!


******************************************************************************************
Please select an option...
1. Log in (or Sign Up)
2. Exit
******************************************************************************************
Please choose one of the options (1 - 2): 1

Loading login interface...
******************************************************************************************
Log in (or Sign Up)
******************************************************************************************
Please enter username: hsyun
Please enter password: asdf
******************************************************************************************
Welcome, hsyun
Please select an option...
1. Display subway line information (Line 1 - 4)
2. Display subway station information
3. Find a path between two subway stations
4. View my recent request
5. Exit
******************************************************************************************
Please cho

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
